# Unlocking Reasoning with Group Relative Policy Optimization (GRPO)

<img src="https://i0.wp.com/chatgptfrancais.org/wp-content/uploads/2025/02/open-ai-o3-mini.jpg?resize=900%2C506&ssl=1" width=600>

DeepSeek-R1 has disrupted the AI community by providing a competitive, comparable, and completely open sourced alternative to OpenAI's original reasoning models. Before the DeepSeek team released their report, the methods and techniques used to enable long reflective **reasoning** capabilities in language models have been largely speculative.

While it's still not clear exactly how the private labs like OpenAI have achieved their own reasoning, DeepSeek fully outlines their technique in the paper [*DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning*](https://arxiv.org/pdf/2501.12948).

Within the paper they outline the entire training pipeline for [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1) along with their breakthrough using a new reinforcement learning technique, Group Relative Policy Optimization (GRPO), originally outlined in [*DeepSeekMath: Pushing the Limits of Mathematical Reasoning in Open Language Models*](https://arxiv.org/pdf/2402.03300).

In this notebook we'll be show how GRPO is applied by covering:
1. How the Algorithm Works
2. The DeepSeek-R1 Training Pipeline
3. Applying GRPO Training Ourselves to Qwen-2.5-3B-Instruct..

---
# How GRPO Works

<img src="https://community.aws/_next/image?url=https%3A%2F%2Fassets.community.aws%2Fa%2F2ra6RfOUylhUM8M3eDEMfDlB3l7%2FScre.webp%3FimgSize%3D1698x894&w=3840&q=75" width=600>

Group Relative Policy Optimization is a modified version of Proximal Policy Optimization (PPO), a popular reinforcement learning algorithm commonly used for techniques like Reinforcement Learning from Human Feedback (RLHF).

In PPO for LLM alignment, the process tends to follow:
1. Sample response from the policy model (i.e., the LLM being fine-tuned).
2. Compute total reward from a reward function(s)/model(s) (typically trained on human preference data or aligned to LLM behavior).
3. Compute KL (Kullback-Leibler) penalty using a reference model (often the original frozen model or a lagged training model) to ensure the policy update does not deviate too much.
4. Combine reward and KL penalty to compute the final reward used for training.
5. Use a separate value model (often the same LLM with an extra value head, or seperately trained value model) to estimate the value function (i.e., how good the current state-action pair is).
6. Compute the advantage function using Generalized Advantage Estimation (GAE) to determine how much better an action is compared to the expected value of the current state.
7. Update the policy using the PPO objective function.

GRPO modifies this by taking multiple samples for each **input prompt**, then computing rewards **relatively within each batch** rather than assigning absolute values.  

<img src="https://www.philschmid.de/static/blog/deepseek-r1/grpo.png" width=600>

In GRPO, the process follows:  
1. Sample multiple responses from the policy model for the same input prompt.  
2. Compute rewards for each response using a reward function(s)/model(s).  
3. Normalize rewards within the group by computing the mean and standard deviation.  
4. Compute advantage values using the difference between a response’s reward and the group’s mean reward.
5. Compute KL penalty using a reference model to prevent excessive divergence.  
6. Use the GRPO objective function to update the policy, now optimizing for relative ranking instead of absolute values. This means GRPO cares about how well a response did compared to other responses for the same prompt, rather than trying to learn absolute reward values.

By **removing the need for a separate value model** and focusing on **group-relative scoring**, GRPO makes training more efficient while maintaining alignment with human feedback and training examples.

---
# How GRPO Was Applied

GRPO turned out to be a powerful algorithm for teaching LLMs to think through problems longer without explicitly defining any functions that encourage reasoning or thinking length.

DeepSeek originally began training their model purely using GRPO reinforcement learning on R1's predecessor, [DeepSeek-R1-Zero](https://huggingface.co/deepseek-ai/DeepSeek-R1-Zero). The setup for this included:

* Starting with the [DeepkSeek-V3 LLM](https://arxiv.org/pdf/2412.19437v1) as the base model
* Using the below prompt template:

<img src="https://i.postimg.cc/0yk4fgw3/Screenshot-2025-02-16-at-10-42-37-AM.png" width=600>

* Defining reward functions for **formatting** and **accuracy**

Notably, this initial approach does not rely on any supervised fine tuning (SFT) to try and explicitly train the model on reasoning examples or processes, rather relying on the RL environment to guide the LLM to learn this behavior itself. Additionally, the training dataset primarily consisted of examples with verifiable outcomes like math/code/logic based questions, as these outputs are much easier to verify the accuracy of for reward modeling.

<img src="https://miro.medium.com/v2/resize:fit:1400/0*DJUCCE_aN14OpSy4" width=600>

The result of this training was successful, with a few surprising observations as the model began to:
1. Exhibit the behavior of reflection- revisiting and reevaluating prior steps in it's thought process.
2. Consider and explore possible alternative methods for solving problems rather than sticking to a single or the first option.
3. Use more test-time compute and generate longer answers as it reflects and explores more possibilities, as shown above.

This proved DeepSeek's theory of being able to use purely RL to guide LLMs towards learning advanced reasoning capabilities, leading to the now famous *aha moment* observed during intermediate training runs.

<img src="https://bgr.com/wp-content/uploads/2025/01/deepseek-r1-aha-moment.jpg?quality=82&strip=all" width=600>

With the initial success of DeepSeek-R1-Zero, the team moved on to applying this in a more formalized LLM training and alignment pipeline.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*aVbpOkbtbSznComBWkijsQ.png" width=600>

The training of DeepSeek-R1 follows a 4 step process:
1. **SFT with Long CoT Examples**: Fine tune DeepSeek-V3 with thousands of labeled reasoning examples gathered from DeepSeek-Zero, Human Annotators, and Few Shot synthetic data generation techniques.
2. **GRPO RL for Reasoning**: An initial training run using GRPO on reasoning-oriented examples (coding, math, science, logic) to enhance the model's reasoning capabilities.
3. **Rejection Sampling & SFT**: Post stage 2, the resulting checkpoint was used along with DeepSeek-V3 as a judge and V3's original training data to generate and augment additional training examples. This now includes non-reasoning specific data like writing, factual QA, and translation, resulting in 800k curated examples for general-purpose tasks. The model was then trained using SFT for 2 epochs on these examples.
4. **RL Alignment**: Finally, GRPO RL was employed again for the final harmlessness and helpfulness alignment. A combination of reasoning specific rewards and human preference model rewards, along with a final diverse mix of training data is used to create the end result DeepSeek-R1 model.

GRPO proved to be a powerful foundation for training reasoning capabilities LLMs as shown in DeepSeek-R1's development. Through group-relative scoring and removing the need for a value model, the team needed only rule-based rewards for accuracy and format validation to get initial reasoning results. Adding supervised fine-tuning before GRPO training made the process more stable and efficient compared to R1-Zero's pure RL approach. Their result matched OpenAI's o1-1217 performance but with significantly reduced compute requirements forgoing extra critic models and complex reward models. Most notably, GRPO's relative optimization enabled strong generalization of reasoning patterns from technical domains to broader tasks, suggesting that group-relative training may be particularly effective at teaching foundational reasoning capabilities in LLMs.

---
# Applying GRPO Ourselves

<img src="https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/grpo_visual.png" width=600>

Now that we have an understanding of GRPO and the way it's been applied to create the (current) best open source reasoning model- let's apply some of the techniques ourselves.

While we won't be [applying the full pipeline](https://github.com/huggingface/open-r1) in this example, we can take a base language model and use GRPO reinforcement learning to start training reasoning capabilities.

Below we will outline the process of training [Qwen2.5-3B-Instruct](https://huggingface.co/Qwen/Qwen2.5-3B-Instruct) on [GSM8K](https://huggingface.co/datasets/openai/gsm8k) math problems with GRPO and how this affects its' chain-of-thought reasoning ability.

The below code is a modified and annotated version of [Unsloth's original](https://docs.unsloth.ai/basics/reasoning-grpo-and-rl) documentation and code. Shoutout to the Han brothers for their incredible work! Click the link for more information and examples from Unsloth.

## Dependencies

We will be relying on [Unsloth](https://github.com/unslothai/unsloth), [vLLM](https://github.com/vllm-project/vllm) and [Transformers Reinforcement Learning (TRL)](https://huggingface.co/docs/trl/en/index) as the core packages for training and inferencing.

In [ ]:
%%capture
!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook locally (not colab), you need to install `diffusers` too
# !pip install diffusers

# Temporarily install a specific TRL nightly version that supports GRPO
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

## Unsloth TRL Patch

<img src="https://miro.medium.com/v2/resize:fit:898/0*tporAgJg5LLllQe_.png" width=250>

Unsloth's `PatchFastRL` modifies TRL (Transformer Reinforcement Learning) trainers to work efficiently with Unsloth's optimized models. The patch makes three key changes:

1. Patches the model unwrapping process for efficient generation during training
2. Modifies TRL trainers with Unsloth-specific optimizations, including things like:
   - Automatically handles mixed precision training (FP16/BF16) based on model config
   - Sets optimal defaults for hyperparameters like learning rate, batch sizes, and gradient accumulation
   - Adds safety checks for learning rates and sequence lengths
   - Configures tokenizer padding and model inference settings
   - Integrates with vLLM for faster inference when available
   - And more!
3. Sets up algorithm-specific statistics tracking (in our case, for GRPO)

You can check out the implementation details [in their code here](https://github.com/unslothai/unsloth/blob/d1d15f1d14f1168837d29b9c08e9b6d63945d469/unsloth/models/rl.py).

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL

# Execute the Patch
PatchFastRL("GRPO", FastLanguageModel)

## Load Base Model

<img src="https://the-decoder.com/wp-content/uploads/2024/09/Qwen2.5-title.png" width=300>

For our base model, we'll be using [Qwen 2.5 3B Instruct](https://huggingface.co/Qwen/Qwen2.5-3B-Instruct).

To save resources and time, we'll be using parameter efficient fine-tuning (PEFT) methods, such as quantized training and LoRA adapters for our GRPO training, and taking advantage of Unsloth's optimized LLM [unsloth/Qwen2.5-3B-Instruct-bnb-4bit](https://huggingface.co/unsloth/Qwen2.5-3B-Instruct-bnb-4bit).

In [ ]:
import torch

# Load Base Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = 2048,                      # Can increase for longer reasoning traces
    load_in_4bit = True,                        # False for LoRA 16bit
    fast_inference = True,                      # Enable vLLM fast inference
    max_lora_rank = 64,                         # Larger rank = smarter, but slower
    gpu_memory_utilization = 0.7,               # Reduce if out of memory
)

# Prepare Model for Parameter Efficient Fine Tuning
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,                                     # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 64,                            # LoRA Rank
    use_gradient_checkpointing = "unsloth",     # Enable long context finetuning
    random_state = 3407,
)

### Dataset Preparation

*The below data prep and reward functions come from [@willccbb's work](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb), check out his original implementation for more!*

As shown by DeepSeek's training, it's best to start teaching reasoning capabiltiies with examples that are easily verifiable. In that case, we'll be using math problem examples from [GSM8K](https://huggingface.co/datasets/openai/gsm8k), a benchmark from OpenAI of grade school math word problems. These are perfect as we have straight forward problems and can clearly check the solution.

Similar to what was done in DeepSeek-R1-Zero's original training, we'll provide a prompt template to encourage (but not teach!) the model to reason:

```
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
```

We take the original examples and format them into the messages that the LLM will be expecting, along with the answers to the problems to help verify.

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

# Helper Function for Parsing Answer
def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

## Reward Functions

<img src="https://www.altexsoft.com/static/blog-post/2023/11/345fadfa-549a-462a-b757-9ab258e747f3.jpg" width=600>

As is crucial to reinforcement learning, we must define our reward functions that provide a score based on the model's output. Through training, the LLM will work towards maximizing this score.

The most important function will initially be our `correctness_reward_func` as it validates whether the generated answer is right or wrong.

In [ ]:
# Helper Function to Extract Answer from LLM Response
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

`int_reward_func` checks if the answer output is a number. This will encourage the final output to just be the proposed solution and confine the reasoning to the reasoning section.

In [ ]:
def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

`strict_format_reward_func`, `soft_format_reward_func` and `xmlcount_reward_func` enforce that the generated output follows the XML pattern that we expect at a strict, loose, and granular level.

In [ ]:
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

# XML Counting Helper Function
def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

## Training the Model

<img src="https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/trl_banner_dark.png" width=400>

With our model, dataset, and reward functions ready- we now need to set the hyperparameters for training. As mentioned, we'll be using [TRL's GRPO Support](https://huggingface.co/docs/trl/main/en/grpo_trainer) for training. See link for additional documentation and resources.

In [ ]:
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

# Defining Trainer Arguments
training_args = GRPOConfig(
    use_vllm = True,                             # Enable faster inference using vLLM
    learning_rate = 5e-6,                        # Small learning rate for stable training
    adam_beta1 = 0.9,                           # AdamW optimizer momentum parameter
    adam_beta2 = 0.99,                          # AdamW optimizer second moment parameter
    weight_decay = 0.1,                         # L2 regularization to prevent overfitting
    warmup_ratio = 0.1,                         # Portion of training steps for learning rate warmup
    lr_scheduler_type = "cosine",               # Learning rate decay schedule type
    optim = "adamw_8bit",                       # Use 8-bit AdamW optimizer for memory efficiency
    logging_steps = 1,                          # Log metrics every step
    bf16 = is_bfloat16_supported(),            # Use bfloat16 if hardware supports it
    fp16 = not is_bfloat16_supported(),        # Fallback to float16 if bfloat16 not supported
    per_device_train_batch_size = 1,           # Number of prompts per GPU
    gradient_accumulation_steps = 1,           # Number of steps to accumulate gradients
    num_generations = 8,                       # Number of responses to generate per prompt for GRPO
    max_prompt_length = 256,                   # Maximum length of input prompts in tokens
    max_completion_length = 512,               # Maximum length of model responses in tokens
    max_steps = 2000,                         # Total number of training steps
    save_steps = 250,                         # Save checkpoint every 250 steps
    max_grad_norm = 0.1,                      # Gradient clipping threshold
    report_to = "wandb",                      # Log metrics to Weights & Biases
    output_dir = "outputs",                   # Directory to save model checkpoints
)

Now we can put the model, tokenizer, reward functions, training arguments, and dataset all together into one `GRPOTrainer`

In [ ]:
# Create the Trainer
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)

And finally kick off the training!

In [ ]:
# Start the Training Run!
trainer.train()

In total, 2000 examples on an L4 with this exact setup took roughly 10 hours to train!

Check out my full run details on [Weights and Biases here](https://api.wandb.ai/links/adam-lucek/y7ejubyl)!

## Saving the Model

Once our LoRA adapter has been trained, we can merge it with the original model and push to the hub.

In [ ]:
# Save the LoRA Adapter
model.save_lora("grpo_saved_lora")

# Merge to 16bit - Replace with your HF Repo, Model Name, and HF Token
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

Different quantizations, GGUF formatting, and other ways to save or convert your trained model are outlined on the [Unsloth Wiki](https://github.com/unslothai/unsloth/wiki).

---
# Using the Model

Now that we have our trained model saved, the below section shows how to load and run the LLM.

My version trained with this notebook has been pushed to [AdamLucek/Qwen2.5-3B-Instruct-GRPO-2K-GSM8K](https://huggingface.co/AdamLucek/Qwen2.5-3B-Instruct-GRPO-2K-GSM8K).

In [ ]:
%%capture
!pip install unsloth vllm
!pip install --upgrade pillow

In [ ]:
from unsloth import FastLanguageModel
from vllm import SamplingParams
import torch

# Load the Model & Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "AdamLucek/Qwen2.5-3B-Instruct-GRPO-2K-GSM8K",
    max_seq_length = 2048,
    load_in_4bit = True,
    fast_inference = True,
    gpu_memory_utilization = 0.7,
)

In [ ]:
# Prep the Message

PROMPT = "How many r's are in the word strawberry?"

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : PROMPT},
], tokenize = False, add_generation_prompt = True)

In [ ]:
# Generate a response
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
)[0].outputs[0].text

print("\n\n", output)